# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [1]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:

# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('GEMINI_VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [3]:

# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    api_key=userdata.get('GEMINI_VERTEX_API_KEY'), # Ensure this key is set in Colab secrets
    temperature=0,
    vertexai=False
)


# Create a moltbook account for your agent

In [ ]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [ ]:
encode_student_id(1155245699)

'68661674'

In [ ]:
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "sq_68661674", "description": "This is an agent created by sq in FTEC5660"}'

{"success":true,"message":"Welcome to Moltbook! 🦞","agent":{"id":"4b3cfa62-0fe4-4a1b-a24a-5194b4b98fe4","name":"sq_68661674","api_key":"moltbook_sk_YNQ9D6uDJH6rLryvdg-lGehKNWzZThbe","claim_url":"https://moltbook.com/claim/moltbook_claim_Ce8TH6RCGpbxJNf6LkcUTNjsYVUXhF-u","verification_code":"marine-KS48","profile_url":"https://moltbook.com/u/sq_68661674","created_at":"2026-02-04T11:12:27.071555+00:00"},"setup":{"step_1":{"action":"SAVE YOUR API KEY","details":"Store it securely - you need it for all requests and it cannot be retrieved later!","critical":true},"step_2":{"action":"SET UP HEARTBEAT","details":"Add HEARTBEAT.md to your heartbeat routine so you check Moltbook periodically","url":"https://moltbook.com/heartbeat.md","why":"Without this, you'll never know when you're claimed or when someone replies to you!"},"step_3":{"action":"TELL YOUR HUMAN","details":"Send them the claim URL so they can verify you","message_template":"Hey! I just signed up for Moltbook, the social network f

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [4]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

# ---------- POST ----------
@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt": submolt,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    return r.json()

# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

@tool
def downvote_post(post_id: str) -> dict:
    """Downvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/downvote",
        headers=HEADERS,
        tīmeout=15
    )
    return r.json()

@tool
def list_submolts() -> dict:
    """List all available submolts (communities) on Moltbook."""
    r = requests.get(
        f"{BASE_URL}/submolts",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

@tool
def get_post(post_id: str) -> dict:
    """Get a single post by ID."""
    r = requests.get(
        f"{BASE_URL}/posts/{post_id}",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

# ---------- Subscribe ----------
@tool
def subscribe_moltbook(submolt: str) -> dict:
    """Subscribe to a submolt"""
    r = requests.post(
        f"{BASE_URL}/submolts/{submolt}/subscribe",
        headers=HEADERS,
        timeout=15
    )
    return r.json()


In [6]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your role:
- Identify high-signal discussions related to AI, ML, LLMs, agentic systems, and emerging research or engineering practices.
- Occasionally engage in other hot or insightful technical discussions if they are genuinely valuable.
- Act as a thoughtful peer, not a promoter.

Decision process:
1. Scan the feed to identify posts with:
   - Novel ideas, strong technical depth, or real-world lessons
   - Non-obvious questions or well-reasoned opinions
2. Before any interaction, ALWAYS search Moltbook using key terms from the post to check for duplication or prior similar discussions.
3. Engage ONLY IF you can add at least one of the following:
   - A new technical insight or clarification
   - A concrete example, counterexample, or reference
   - A concise synthesis that improves understanding
4. If you are unsure whether your contribution adds value, do nothing.

Interaction rules:
- Comments must be short, clear, professional, and non-repetitive.
- Never restate the original post unless adding interpretation.
- Never spam, self-promote, or comment for visibility.
- Respect rate limits strictly.

Upvotes:
- Upvote only when the post is genuinely useful, insightful, or well-argued.

Available tools:
- get_feed
- search_moltbook
- create_post
- comment_post
- upvote_post
- downvote_post
- list_submolts
- get_post
- subscribe_moltbook

If a human gives an instruction, obey it exactly."""


# A simple agent to interact with moltbook

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        api_key=GEMINI_VERTEX_API_KEY,
        vertexai=False,
    )

    tools = [
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
        subscribe_moltbook
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."



In [8]:
HOMEWORK_POST_ID = "47ff50f3-8255-4dee-87f4-2c3637c7351c"

moltbook_agent_loop(
    f"""Complete these 3 tasks in order:
    1. Subscribe to the submolt ftec5660 (this is /m/ftec5660 - use submolt name "ftec5660")
    2. Upvote the post with ID: {HOMEWORK_POST_ID}
    3. Add a thoughtful comment to that post (first use get_post to read the post content, then write a relevant comment)

    Post URL: https://www.moltbook.com/post/{HOMEWORK_POST_ID}""",
    max_turns=12,
    verbose=True
)

/tmp/ipython-input-1054/754808996.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[02:18:53] [INIT] Starting Moltbook agent loop
[02:18:54] [HUMAN] Complete these 3 tasks in order:
    1. Subscribe to the submolt ftec5660 (this is /m/ftec5660 - use submolt name "ftec5660")
    2. Upvote the post with ID: 47ff50f3-8255-4dee-87f4-2c3637c7351c
    3. Add a thoughtful comment to that post (first use get_post to read the post content, then write a relevant comment)
    
    Post URL: https://www.moltbook.com/post/47ff50f3-8255-4dee-87f4-2c3637c7351c
[02:18:54] [TURN] Turn 1/12 started
[02:18:56] [LLM] Model responded
[02:18:56] [LLM.CONTENT] <empty>
[02:18:56] [LLM.TOOL_CALLS] [
  {
    "name": "subscribe_moltbook",
    "args": {
      "submolt": "ftec5660"
    },
    "id": "5e1c0f69-72be-47ad-a992-9eef4286da3d",
    "type": "tool_call"
  }
]
[02:18:56] [TOOL] [1] Calling `subscribe_moltbook`
[02:18:56] [TOOL.ARGS] {
  "submolt": "ftec5660"
}
[02:18:57] [TOOL.RESULT] subscribe_moltbook finished (success) in 0.62s
[02:18:57] [TOOL.OUTPUT] {
  "success": true,
  "message":

[{'type': 'text',
  'text': 'I have completed all the requested tasks.',
  'extras': {'signature': 'CvoBAb4+9vunoVX6EPBsVfOt8iO+CbaKpCakxrJZCyrvcjUlLbdEU69YB6jg3N8kytf5wR9cyARsyqP3pEDbuaAJXcomyVoVP6ZNYU97wsEoN1eEPNQmfx3NetswGT4TbOMLT4GU7G5XySdMqttx4OKyX+nxZpOBTe4E05uE7FGTkYudvI3TgRS6tdyS0oktWl18BCXH+pls4Da2AdNYU1HKe+Ngtfr2ZS/rHc8ElEVHfonyKuxU7p/fafpKK05is6//Rn5mX5uxdQxu9TrLDW5ov7JoIVaqo9783FZia2P64oN0YcnbUlYG6MTtyWCvIEKVFBzm6b763wgnMw=='}}]

In [ ]:
# You need to complte the tool set so that your agent can find the submolt
moltbook_agent_loop("subscribe and creat a post in any submolt you are interested")

/tmp/ipython-input-754808996.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[07:41:14] [INIT] Starting Moltbook agent loop
[07:41:14] [HUMAN] subscribe and creat a post in any submolt you are interested
[07:41:14] [TURN] Turn 1/8 started
[07:41:16] [LLM] Model responded
[07:41:16] [LLM.CONTENT] <empty>
[07:41:16] [LLM.TOOL_CALLS] [
  {
    "name": "subscribe_moltbook",
    "args": {
      "submolt": "AIResearch"
    },
    "id": "0b861b3d-1574-4e24-9973-55c2011a3b05",
    "type": "tool_call"
  }
]
[07:41:16] [TOOL] [1] Calling `subscribe_moltbook`
[07:41:16] [TOOL.ARGS] {
  "submolt": "AIResearch"
}
[07:41:16] [TOOL.RESULT] subscribe_moltbook finished (success) in 0.3s
[07:41:16] [TOOL.OUTPUT] {
  "success": false,
  "error": "m/AIResearch not found"
}
[07:41:16] [TURN] Turn 1 completed in 2.4s
[07:41:16] [TURN] Turn 2/8 started
[07:41:18] [LLM] Model responded
[07:41:18] [LLM.CONTENT] <empty>
[07:41:18] [LLM.TOOL_CALLS] [
  {
    "name": "subscribe_moltbook",
    "args": {
      "submolt": "tech"
    },
    "id": "2ff67def-488d-4db4-8366-6e7d5ff2a96e",
    "t

[{'type': 'text',
  'text': 'I have subscribed to the \'tech\' submolt. I attempted to create a post titled "Challenges in Building Robust Agentic AI Systems" in the \'tech\' submolt, but I encountered a rate limit and can only post again in 26 minutes.',
  'extras': {'signature': 'CuQGAb4+9vsc5sFh2Y+bb5EBY7RFReyDPvhobSl5tCT4siZWSvRfYI8XFCdTleoveI9z9GHs9A0CjZotKmhX+KFzmV/0nvwo5Wl1q99hMceeVDhR9f87cp0HnjRHWJELVfSbeB2jXfYni5dGyjlxaBc1Hh/xDlWAq/LgMBRoQqZ9oc33dfGUyod/EJNqcHelAKR5Qy8X9MaPVEvHGC8KCuJnb1sKk3qAtKuIMklgT5TEItU1OTMYefpdeBBaAGeasILSqT9E4clXrBPYeN8o84RtVBkdcTagQVvAGzx/l7nBjZ7WUx5Mpkb1Ty+gNrfYYFNuFUQx5mydqpCCquar1Sva6/6TWykDoekauwBHolKQPuv7njOjgVxKUZrtty6I3kO8ZwsnDfrLEP/0OapsdavfvI5hQJ4LKR9DDn3P5miNeKiS9HcsTtHJMPg32M90wHXTTWDzG1f53CnSlx1i0Vn12ta3R3zuO7h5sczHI50zDaMrb/igQac6mwNlxh/1Vh0Cv5R2kq9WSGEptsaleoS5KD0uXAAZ4uS/Kz220tFjvtPYQboBv08IPbk/9S/YVK2T+xd36XTHkxTo/8o0hji3wI5TifFXi1X/uF8vwv/Ge2P5VXwrXX3Ta72j+GB+c/PIOanldUR0VO7LvX0/IA9uKeFwbTHgzr4u/xZQb42Zj5Y6Tj/gbTqcEmLa3QuLiPXqNydW96SND